### 3. GenAi-task

In [1]:
import pandas as pd
import os
import random
from dotenv import load_dotenv

from huggingface_hub import login

In [2]:
# Load environment variables from the .env file
load_dotenv()

# Access the Hugging Face token
hf_token = os.getenv("HF_TOKEN")

# Use the token to authenticate with Hugging Face
login(token=hf_token)

In [3]:
project_path = 'dataset/interim/cleaned_with_sentiment_numeric.csv'
categories_path = 'dataset/interim/amazon_review_categories.csv'

In [4]:
df = pd.read_csv(project_path)

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/905003903.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(project_path)


In [5]:
df_categories = pd.read_csv(categories_path)

In [6]:
df_categories.head()

,category,dominant_topic,label
0,"Fire Tablets,Tablets,Computers & Tablets,All T...",2,tablets
1,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",3,home
2,"Back To College,College Electronics,College Tv...",4,Topic 4
3,"Walmart for Business,Office Electronics,Tablet...",2,tablets
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",2,tablets


In [7]:
#clean names:
# Clean product names: remove special characters, extra commas, and strip whitespace
df['name'] = df['name'].replace({r'\r': ' ', r'\n': ' ', ',,': ','}, regex=True).str.strip()

# Remove any remaining multiple spaces
df['name'] = df['name'].replace(r'\s+', ' ', regex=True)

In [8]:
df['name_clean'] = df['name'].str.split(',').str[0]

In [9]:
df_all = pd.merge(df,df_categories[['category','label']],how='left',left_on='categories',right_on='category')

In [12]:
grouped_reviews = df_all.groupby(['label','name_clean'])['reviews.text'].apply(list).reset_index()

In [13]:
# Assume the column with lists is named 'list_column'
# Create a new column 'list_length' that stores the number of items in each list
grouped_reviews['nr_reviews'] = grouped_reviews['reviews.text'].apply(len)

In [14]:
ratings_helper = pd.DataFrame(df_all.groupby('name_clean')['sentiment'].mean()).reset_index()

In [15]:
df_final = pd.merge(grouped_reviews,ratings_helper,on='name_clean')

In [16]:
df_final_filtered = df_final[df_final['nr_reviews']>=5]

In [17]:
df_final_filtered.head(2)

,label,name_clean,reviews.text,nr_reviews,sentiment
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088


In [20]:
df_sorted = df_final_filtered.sort_values(by=['label', 'sentiment'], ascending=[True, False])

# Group by 'category' and select the top 3 rows per group
top_3_per_category = df_sorted.groupby('label').head(3)
last_3_per_category = df_sorted.groupby('label').tail(3)

In [21]:
top_3_per_category['cat'] = '5 Star'
last_3_per_category['cat'] = 'Fiasko'

top_categories = pd.concat([top_3_per_category,last_3_per_category])
top_categories = top_categories.drop_duplicates(subset=['name_clean'])

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/421275397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_3_per_category['cat'] = '5_star'
/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/421275397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_3_per_category['cat'] = 'fiasko'


,label,name_clean,reviews.text,nr_reviews,sentiment,cat
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088,5_star
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039,5_star
6,home,Amazon Fire Hd 6 Standing Protective Case(4th ...,[My family really enjoys the Amazon Echo. We h...,7,2.000000,5_star
13,home,Kindle Dx Leather Cover,[Purchased the Echo as a bridal shower gift. M...,9,2.000000,5_star
4,home,Amazon 9W PowerFast Official OEM USB Charger a...,[Alexa is the greatest product ever I use it a...,36,1.972222,5_star
17,tablets,Amazon - Fire 16GB (5th Gen,"[I love my Kindle Voyage, however it does not ...",5,2.000000,5_star
20,tablets,Amazon Echo and Fire TV Power Adapter,[This is the first e-reader I've gotten and I ...,8,2.000000,5_star
22,tablets,Amazon Fire Kids Edition Tablet,[The text is sharp and paper like in well lit ...,6,2.000000,5_star
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088,fiasko
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039,fiasko


In [92]:
import random

# Function to randomly select 9 reviews from the list of reviews
def select_random_reviews(reviews_list):
    # If there are fewer than 3 reviews, return them all
    if len(reviews_list) <= 20:
        return reviews_list
    # Otherwise, return 3 random reviews
    return random.sample(reviews_list, 20)

In [93]:
top_categories['reviews.text'] = top_categories['reviews.text'].apply(select_random_reviews)

In [94]:
for f in top_categories['reviews.text']:
    print(f)

['Stay away from the certified refurbished Amazon Fire TV. I received two Certified Refurbished defective fire TVs. The first the screen looked half corrupted, see the pictures. The second Certified Refurbished Amazon fire tv I received would not boot up, it was stuck in an infinite loop where I would only see the amazon logo for 10 seconds then it would boot up again.If anything, Amazon needs to re-visit its refurbishment process and quality control of the refurbished products they let get in consumer hands.Update 11/28/16: I purchased a new Fire TV during the BF sale and I have had no issues so far :).', "I seriously wonder why it took me to get FireTV for so long. I originally had a Roku XS and then I stupidly bought a Roku 4. Both devices were all right but neither could handle 5.1 audio from Netflix on my Vizio sound bar (essentially your device must be cable of handling Dolby Digital Plus and nothing less using Roku). I finally decided to give FireTV a try and I'm amazed. Not onl

## GenAi


#### SUMMARIZATION

In [22]:

# Function to randomly select  reviews and generate a summary
def summarize_reviews(reviews_list, num_reviews=15):
    # If there are fewer reviews than num_reviews, use all reviews
    if len(reviews_list) < num_reviews:
        selected_reviews = reviews_list
    else:
        # Randomly select 5 reviews from the list
        selected_reviews = random.sample(reviews_list, num_reviews)
    
    # Convert the selected reviews to a single string
    reviews_text = " ".join(selected_reviews)
    
    # Generate a summary
    summary = summarizer(reviews_text, max_length=150, min_length=50, do_sample=False)
    
    # Return the summary text
    return summary[0]['summary_text']

In [23]:
import torch
from transformers import pipeline
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Load the summarization pipeline (you can use BART or another model)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.backends.mps.is_available() else -1)

In [25]:
top_categories['summary'] = top_categories['reviews.text'].apply(summarize_reviews)

    # Save the DataFrame to a new CSV
top_categories.to_csv('dataset/interim/products_with_summaries.csv', index=False)

    # Display the updated DataFrame with summaries

Your max_length is set to 150, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)


In [27]:
top_categories[['name_clean', 'summary','cat','label']]

,name_clean,summary,cat,label
1,Echo (White),This is one of the best purchases I have made ...,5_star,Topic 4
0,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,The image quality (movies and games) is excell...,5_star,Topic 4
6,Amazon Fire Hd 6 Standing Protective Case(4th ...,The Amazon Echo is a great way to control your...,5_star,home
13,Kindle Dx Leather Cover,The Amazon Echo has very good voice recognitio...,5_star,home
4,Amazon 9W PowerFast Official OEM USB Charger a...,"Alexa is great for keeping shopping lists,cont...",5_star,home
17,Amazon - Fire 16GB (5th Gen,"I love my Kindle Voyage, however it does not h...",5_star,tablets
20,Amazon Echo and Fire TV Power Adapter,This is the first e-reader I've gotten and I L...,5_star,tablets
22,Amazon Fire Kids Edition Tablet,Slim and lightweight when not using a case whi...,5_star,tablets
1,Echo (White),This is one of the best purchases I have made ...,fiasko,Topic 4
0,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,The image quality (movies and games) is excell...,fiasko,Topic 4


### Cerate Posts:

# Generate blogposts blogposts:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the tokenizer and model
model_name = "EleutherAI/gpt-neo-1.3B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the correct device

# Initialize the pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

def generate_blog_post(product_name, summary):
    prompt = f"Write a blog post about the product '{product_name}' based on the following review summary: {summary}"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move inputs to the correct device
    input_ids = inputs["input_ids"]
    
    # Generate text with attention_mask and max_length control
    outputs = model.generate(
        input_ids, 
        attention_mask=inputs["attention_mask"],  # Add attention_mask to inputs
        max_length=500, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2, 
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # Set pad_token_id explicitly
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Split the text at the first two new lines (\n\n) and keep the part after the split
    blog_post = generated_text.split("\n\n", 1)[-1].strip()  # Ensure you only keep the blog post text

    return blog_post

# Create a dictionary to store blog posts for each category
blog_posts = {}

# Iterate over the rows in your dataframe
for idx, row in top_categories.iterrows():
    
    category = row['label']
    product_name = row['name_clean']
    summary = row['summary']
    cat = row['cat']
    print(product_name,cat)
    # Generate a blog post
    blog_post = generate_blog_post(product_name, summary)
    
    # Store the blog post in the dictionary
    if category not in blog_posts:
        blog_posts[category] = []
    blog_posts[category].append(blog_post)

# Print a sample blog post for a category
for category, posts in blog_posts.items():
    print(f"Blog post for category '{category}':\n{posts[0]}\n")


Echo (White) 5_star
Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue 5_star
Amazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release) 5_star
Kindle Dx Leather Cover 5_star
Amazon 9W PowerFast Official OEM USB Charger and Power Adapter for Fire Tablets and Kindle eReaders 5_star
Amazon - Fire 16GB (5th Gen 5_star
Amazon Echo and Fire TV Power Adapter 5_star
Amazon Fire Kids Edition Tablet 5_star
Echo (White) fiasko
Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue fiasko
Fire Tablet fiasko
New Amazon Kindle Fire Hd 9w Powerfast Adapter Charger + Micro Usb Angle Cable fiasko


#### export results

In [100]:
# Connect to the SQLite database (it will create the database file if it doesn't exist)
db_path = "flask_project/data/blogposts.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Drop the table if it already exists to ensure you're recreating it
cursor.execute('''
DROP TABLE IF EXISTS blogposts
''')

# Create the table again (this is essentially replacing the old table)
cursor.execute('''
CREATE TABLE blogposts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    category TEXT,
    product_name TEXT,
    blogpost TEXT
)
''')

# Commit the table creation
conn.commit()

# Iterate over the rows in your dataframe and generate blog posts for each row
for idx, row in top_categories.iterrows():
    category = row['label']
    product_name = row['name_clean']
    summary = row['summary']
    
    # Generate a blog post for each product
    blog_post = generate_blog_post(product_name, summary)
    
    # Insert each blog post into the SQLite table
    cursor.execute('''
        INSERT INTO blogposts (category, product_name, blogpost)
        VALUES (?, ?, ?)
    ''', (category, product_name, blog_post))

# Commit the inserts
conn.commit()

# Close the connection
conn.close()

print("Blog posts successfully saved to SQLite database.")


Blog posts successfully saved to SQLite database.
